In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier

In [34]:
TRAIN_FILE = 'data/train.csv'
TEST_FILE = 'data/test.csv'
REVEALED_TEST_FILE = 'data/revealed_test.csv'
CENSUS_FILE = 'data/census_starter.csv'

In [35]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def one_hot(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    for each in columnsToEncode:
        df=pd.concat([df,pd.get_dummies(df[each],prefix=each, drop_first=True)],axis=1).drop([each],axis=1)
    return df

def fill_na(df):
    columnsToEncode = list(df.select_dtypes(include=['category','object']))
    for each in columnsToEncode:
        df[each] = df[each].fillna(df[each].mode().iloc[0])
    df = df.fillna(df.median())
    return df

In [50]:
train = load_data(TRAIN_FILE)
test = load_data(REVEALED_TEST_FILE)
census = load_data(CENSUS_FILE)
all_data = train.append(test)
NUM_TRAIN = len(train)
NUM_TEST = len(test)

/var/folders/sz/96l2wmtn19l4dg18frjfvmfm0000gn/T/ipykernel_1171/1582917920.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = train.append(test)


(122265, 6270)

# Data Cleaning

In [54]:
all_data = all_data.drop(['row_id'], axis = 1)

# Feature Engineering

In [57]:
# One hot and imputation
all_data = fill_na(all_data)
all_data = one_hot(all_data)
all_data = all_data[~all_data.isin([np.nan, np.inf, -np.inf]).any(1)]
train = all_data[:NUM_TRAIN]
test = all_data[NUM_TRAIN:]
# train = fill_na(train)
# test = fill_na(test)
# train = one_hot(train)
# test = one_hot(test)
# train = train[~train.isin([np.nan, np.inf, -np.inf]).any(1)]
# test = test[~test.isin([np.nan, np.inf, -np.inf]).any(1)]

/var/folders/sz/96l2wmtn19l4dg18frjfvmfm0000gn/T/ipykernel_1171/3301010539.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.median())


# Train

In [60]:
X_train = train.drop(['microbusiness_density'], axis = 1)
y_train = train['microbusiness_density']
X_test = test.drop(['microbusiness_density'], axis = 1)
y_test = test['microbusiness_density']

In [61]:
N_ESTIMATORS = 200
MAX_DEPTH = 5
SUBSAMPLE = 0.8
LEARNING_RATE = 0.01

In [ ]:
gradient_booster = GradientBoostingRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, subsample=SUBSAMPLE, learning_rate=LEARNING_RATE)
gradient_booster.fit(X_train, y_train)

# Inference

In [44]:
test_predictions = gradient_booster.predict(X_test)

/Users/afu/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- first_day_of_month_2022-12-01
Feature names seen at fit time, yet now missing:
- first_day_of_month_2019-09-01
- first_day_of_month_2019-10-01
- first_day_of_month_2019-11-01
- first_day_of_month_2019-12-01
- first_day_of_month_2020-01-01
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 1923 features, but GradientBoostingRegressor is expecting 1960 features as input.

In [ ]:
testauc = roc_auc_score(y_test, test_predictions)
print('AUC is', auc)
fpr, tpr, _ = roc_curve(y_test,  test_predictions)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()_predictions = gradient_booster.predict_proba(X_test)[:,1]